## <center>Сложные соединения<center>

Интересующие нас данные хранятся в таблицах city, customer, driver, shipment, truck. Давайте внимательно их изучим. Ниже представлена ER-диаграмма (от англ. entity-relation, дословно — «сущность-связь»), которая отображает существующие связи между отдельными таблицами.

![](images/relations.jpg)

#### **Задание 1.1**

Укажите название города с максимальным весом единичной доставки.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.city_name  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.shipment s  
&ensp;&ensp;&ensp;&ensp;**NATURAL JOIN** sql.city c  
**GROUP BY**  
&ensp;&ensp;&ensp;&ensp;c.city_id,  
&ensp;&ensp;&ensp;&ensp;s.weight  
**HAVING** s.weight = **MAX**(s.weight)  
**ORDER BY** s.weight **DESC**  
**LIMIT** 1

 Сколько различных производителей грузовиков перечислено в таблице truck?

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;**COUNT**(**DISTINCT** t.make)  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.truck t